# Linear Solver Performance: Constant Coefficient Poisson, Single Core

### Part 1, Benchmark Setup and Execution

In [ ]:
#r "BoSSSpad.dll"
//#r "../../../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [ ]:
using BoSSS.Application.SipPoisson;

In [ ]:
ExecutionQueues

In [ ]:
var myBatch = GetDefaultQueue();
myBatch

In [ ]:
string WFlowName = "run0";
wmg.Init("LinslvPerf_ConstPoissonMpi1");
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

## Utility definitions

In [ ]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
}

## Init grids and save to database

In [ ]:
wmg.Grids

Create meshes in various resolutions:
- domain $\Omega = (0,10) \times (-1,1) \times (-1,1)$; 
- a Dirichlet boundary is set for $x = 0$, everywhere else Neumann boundaries are assumed; 
  these are typically more challenging for a linear solver than Dirichlet boundaries.
- in order to make the problem a bit challenging, a non-equidistant spacing for $y$ and $z$ coordinate are used
- the domain shape is reminiscent of a typical grid for the simulation of channel flows, with exception of the boundary conditions

In [ ]:
int[] Resolutions_3D = new int[] { 4, 8, 16, 24, 32, 48, 64 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(0, 10, Res*5 + 1);
    double[] yNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    double[] zNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "_J" + J);
    
    //grids[cnt] = null;
    //foreach(IGridInfo grd in wmg.Grids) { 
    //    bool check = grd.Name.Contains(string.Format("_J"+ J));
    //    if(check) {
    //      grids[cnt] = grd;
    //    }
    //}
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(string.Format("_J"+ J))); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null) { // 
        Console.WriteLine($"Creating grid with {J} = {Res*5}x{Res}x{Res} cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
    
        g.DefineEdgeTags(delegate (double[] X) {
            if(Math.Abs(X[0] - 0.0) <= 1.0e-6)
                return BoundaryType.Dirichlet.ToString();
            else
                return BoundaryType.Neumann.ToString();
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

In [ ]:
grids

## Setup Control Object for a Solver Run

### Values/Formulas for the Values of RHS and Boundary Conditions

In [ ]:
string nl = System.Environment.NewLine;
string math_expressions_code = 
"static class InitialValues { " +
"    public static double RHS(double[] X) { " + nl +
"        return -Math.Sin(X[0]); " + nl +
"    } " + nl +
"     " + nl +
"    public static double DirichletBC(double[] X) { " + nl +
"        return 0.0; " + nl +
"    } " + nl +
"     " + nl +
"    public static double NeumannBC(double[] X) { " + nl +
"       if(Math.Abs(X[1] - 1.0) < 1.0e-8 || Math.Abs(X[1] + 1.0) < 1.0e-8) " + nl +
"           return 0; " + nl +
"       if(X.Length > 2 && (Math.Abs(X[2] - 1.0) < 1.0e-8 || Math.Abs(X[2] + 1.0) < 1.0e-8)) " + nl +
"           return 0; " + nl +
"     " + nl +
"       return Math.Cos(10.0); " + nl +
"   } " + nl +
"} " + nl;

In [ ]:
var RHS_value = new Formula("InitialValues.RHS", false, math_expressions_code);
var NeumannBC_value = new Formula("InitialValues.NeumannBC", false, math_expressions_code);
var DirichletBC_value = new ConstantValue(0.0);

Brief test of the boundary values:

In [ ]:
RHS_value.Evaluate(new Vector(0.2, 0.1), 0)

In [ ]:
NeumannBC_value.Evaluate(new Vector(0.2, 0.1), 0)

### Setup of Parameter Study

Polynomial degrees to test:

In [ ]:
//int[] PolyDegS = new int[] {2, 3, 4, 5, 6};
int[] PolyDegS = new int[] {2,3,5}; // reduced options for dev.

Solvers which we want to instrument:

In [ ]:
using BoSSS.Solution.AdvancedSolvers;

In [ ]:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.classic_pardiso, 
    //LinearSolverCode.classic_mumps, 
    LinearSolverCode.exp_Kcycle_schwarz,
    LinearSolverCode.exp_gmres_levelpmg,
}; 

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [ ]:
int MaxDof = 5000000; // 5 Million

Loop over all combinations of parameters and define a control object for each combo:

In [ ]:
List<SipControl> controls = new List<SipControl>();
foreach(LinearSolverCode solver_name in solver_nameS) {
foreach(int k in PolyDegS) {   
foreach(IGridInfo grd in grids) {
    int Np = Utils.Np(k);
    int J  = grd.NumberOfCells;
    if(J*Np > MaxDof)
        continue;
    
    var ctrl = new SipControl();
    controls.Add(ctrl);
       
    
    string caseName = string.Format("J{0}_k{1}_{2}", J, k, solver_name);
    Console.WriteLine("setting up: " + caseName);

    ctrl.SetGrid(grd);
    ctrl.savetodb = true;
    ctrl.SetDGdegree(k);

    ctrl.LinearSolver.NoOfMultigridLevels  = int.MaxValue;
    ctrl.LinearSolver.TargetBlockSize      = Math.Min(J*Np-1,10000);
    ctrl.LinearSolver.ConvergenceCriterion = 1e-8;
    
    ctrl.InitialValues.Add("RHS", RHS_value);
    ctrl.AddBoundaryValue(BoundaryType.Dirichlet.ToString(), "T", DirichletBC_value);
    ctrl.AddBoundaryValue(BoundaryType.Neumann.ToString(), "T", NeumannBC_value);
    
    ctrl.SessionName = "SIP_" + caseName;
    
    ctrl.GridPartType = BoSSS.Foundation.Grid.GridPartType.METIS;
    //ctrl.GridPartType = BoSSS.Foundation.Grid.GridPartType.directHilbert;
}
}
}

## Launch Jobs

In [ ]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName);
    ctrl.RunBatch(myBatch); 
}

In [ ]:
//foreach(var ctrl in controls) {
//    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
//    var aJob = new Job(ctrl.SessionName, typeof(SipPoissonMain));
//    aJob.SetControlObject(ctrl);
//    ((SlurmClient)myBatch).SlurmAccount = "project01217";  // Jens 
//    aJob.NumberOfMPIProcs = 1;
//    aJob.ExecutionTime    = "02:00:00";
//    //aJob.HHLR_project             = "project01217";
//    aJob.MemPerCPU                = "50000";
//    aJob.UseComputeNodesExclusive = true;
//    aJob.Activate(myBatch);
//}

In [ ]:
/*
int cnt   = 0;
int iCore = -1;
foreach(var ctrl in controls) {

    if(cnt % JobsWithSameCore == 0)
        iCore++;
         
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    var aJob = new Job(ctrl.SessionName, typeof(SipPoissonMain));
    aJob.SetControlObject(ctrl);
    ((SlurmClient)ExecutionQueues[1]).SlurmAccount = "project01217";  // Jens 
    aJob.NumberOfMPIProcs         = core_sweep[iCore];
    aJob.ExecutionTime            = "02:00:00";
    //aJob.HHLR_project             = "project01217";
    aJob.MemPerCPU                = "2500";
    aJob.UseComputeNodesExclusive = true;
    aJob.Activate(myBatch);
    cnt++;
}
*/

In [ ]:
//foreach(var j in wmg.AllJobs.Values)
//    j.DeleteOldDeploymentsAndSessions();

In [ ]:
wmg.AllJobs

In [ ]:
wmg.AllJobs.First().Value.Stdout